In [2]:
setwd("~/Documents/Upd-Germline")
source("_targets.R")

Loading required package: viridisLite


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'glmGamPoi'


The following object is masked from 'package:dplyr':

    vars



Attaching package: 'purrr'


The following object is masked from 'package:magrittr':

    set_names


Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: 'matrixStats'


The following object is masked from 'package:dplyr':

    count



Attaching package: 'MatrixGenerics'


The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, c

## Histone mark profile plot - local maxima and their significance to the "off" genes t-test

In [3]:
tar_load(matches("^chic.heatmap.tss_H3K(4|27|9)_(Germline|Somatic)_CN_chr"))
tar_load(c(quartile.factor_Germline, quartile.factor_Somatic))

In [6]:
all.equal(
  sort(match(rownames(chic.heatmap.tss_H3K27_Somatic_CN_chr), names(quartile.factor_Somatic))),
  seq_along(quartile.factor_Somatic)
)

[1] TRUE

In [64]:
data <- tibble(
  cross_join(
    tibble(
      celltype = c("Germline", "Germline", "Germline", "Somatic", "Somatic", "Somatic"),
      panel = c("H3K4me3", "H3K27me3", "H3K9me3", "H3K4me3", "H3K27me3", "H3K9me3"),
      heatmap = list(
        chic.heatmap.tss_H3K4_Germline_CN_chr,
        chic.heatmap.tss_H3K27_Germline_CN_chr,
        chic.heatmap.tss_H3K9_Germline_CN_chr,
        chic.heatmap.tss_H3K4_Somatic_CN_chr,
        chic.heatmap.tss_H3K27_Somatic_CN_chr,
        chic.heatmap.tss_H3K9_Somatic_CN_chr
      )
    ),
    tibble(
      rowname = c("low", "medium", "high"),
      level = c("Q2", "Q3", "Q4"),
    ),
  ),
  quant = mapply(
    \(celltype, heatmap) get(str_glue("quartile.factor_", celltype))[rownames(heatmap)],
    celltype,
    heatmap,
    SIMPLIFY=FALSE
  ),
  x = mapply(
    \(level, heatmap, quant) colnames(heatmap)[
      which.max(
        colMeans(heatmap[quant == level, 1:1000], na.rm=T) %>%
          replace(
            c(
              1,
              1 + which(
                diff(diff(colMeans(heatmap[quant == level, 1:1000], na.rm=T))) >= -1e-4
              ),
              1000
            ),
            0
          )
      )
    ],
    level,
    heatmap,
    quant
  ),
  ttest = mapply(
    \(heatmap, quant, level, x) t.test(
      heatmap[quant == level, x],
      heatmap[quant == "Q1", x]
    ),
    heatmap,
    quant,
    level,
    x,
    SIMPLIFY=FALSE
  ),
  t = sapply(ttest, \(obj) obj$statistic["t"]),
  p.adjust = p.adjust(
    sapply(
      ttest,
      \(obj) obj$p.value
    )
  ),
  signif = structure(
    cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
    levels = c("****", "***", "**", "*", ""),
    class = "factor"
  )
) %>%
  subset(select=-c(heatmap,quant,ttest)) %>%
  print()

# A tibble: 18 x 8
   celltype panel    rowname level x           t  p.adjust signif
   <chr>    <chr>    <chr>   <chr> <chr>   <dbl>     <dbl> <fct> 
 1 Germline H3K4me3  low     Q2    131    24.1   1.51e-114 "****"
 2 Germline H3K4me3  medium  Q3    132    37.4   9.17e-242 "****"
 3 Germline H3K4me3  high    Q4    132    40.1   4.40e-271 "****"
 4 Germline H3K27me3 low     Q2    133    -2.82  1.92e-  2 "*"   
 5 Germline H3K27me3 medium  Q3    133    -1.93  1.59e-  1 ""    
 6 Germline H3K27me3 high    Q4    -30    -1.49  2.71e-  1 ""    
 7 Germline H3K9me3  low     Q2    -14     4.37  6.35e-  5 "****"
 8 Germline H3K9me3  medium  Q3    -10     6.52  6.64e- 10 "****"
 9 Germline H3K9me3  high    Q4    -10     5.29  9.92e-  7 "****"
10 Somatic  H3K4me3  low     Q2    308     9.31  1.70e- 19 "****"
11 Somatic  H3K4me3  medium  Q3    134    10.7   2.91e- 25 "****"
12 Somatic  H3K4me3  high    Q4    135     8.96  3.91e- 18 "****"
13 Somatic  H3K27me3 low     Q2    483     5.24  1.16e-  

In [66]:
tar_load(
  c(
    chic.heatmap.tss.nucleosome_H3K27_Germline_CN_chr,
    chic.heatmap.tss.nucleosome_H3K27_Somatic_CN_chr
  )
)

In [68]:
data <- tibble(
  cross_join(
    tibble(
      celltype = c("Germline", "Somatic"),
      heatmap = list(
        chic.heatmap.tss.nucleosome_H3K27_Germline_CN_chr,
        chic.heatmap.tss.nucleosome_H3K27_Somatic_CN_chr
      )
    ),
    tibble(
      rowname = c("low", "medium", "high"),
      level = c("Q2", "Q3", "Q4"),
    ),
  ),
  quant = mapply(
    \(celltype, heatmap) get(str_glue("quartile.factor_", celltype))[rownames(heatmap)],
    celltype,
    heatmap,
    SIMPLIFY=FALSE
  ),
  x = mapply(
    \(level, heatmap, quant) colnames(heatmap)[
      which.max(
        colMeans(heatmap[quant == level, 1:1000], na.rm=T) %>%
          replace(
            c(
              1,
              1 + which(
                diff(diff(colMeans(heatmap[quant == level, 1:1000], na.rm=T))) >= -1e-4
              ),
              1000
            ),
            0
          )
      )
    ],
    level,
    heatmap,
    quant
  ),
  ttest = mapply(
    \(heatmap, quant, level, x) t.test(
      heatmap[quant == level, x],
      heatmap[quant == "Q1", x]
    ),
    heatmap,
    quant,
    level,
    x,
    SIMPLIFY=FALSE
  ),
  t = sapply(ttest, \(obj) obj$statistic["t"]),
  p.adjust = p.adjust(
    sapply(
      ttest,
      \(obj) obj$p.value
    )
  ),
  signif = structure(
    cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
    levels = c("****", "***", "**", "*", ""),
    class = "factor"
  )
) %>%
  subset(select=-c(heatmap,quant,ttest)) %>%
  print()

# A tibble: 6 x 7
  celltype rowname level x         t  p.adjust signif
  <chr>    <chr>   <chr> <chr> <dbl>     <dbl> <fct> 
1 Germline low     Q2    131    29.9 7.84e-168 ****  
2 Germline medium  Q3    133    47.1 0         ****  
3 Germline high    Q4    134    46.5 0         ****  
4 Somatic  low     Q2    129    34.1 4.06e-208 ****  
5 Somatic  medium  Q3    131    42.2 2.22e-290 ****  
6 Somatic  high    Q4    133    38.2 1.02e-248 ****  


## Histone mark at the gene midpoint - t-test

In [73]:
tar_load(matches("^chic.heatmap.paneled_H3K(4|27)_(Germline|Somatic)_CN_chr"))

In [79]:
data <- tribble(
  ~mark, ~germline, ~somatic,
  "H3K4me3", chic.heatmap.paneled_H3K4_Germline_CN_chr, chic.heatmap.paneled_H3K4_Somatic_CN_chr,
  "H3K27me3", chic.heatmap.paneled_H3K27_Germline_CN_chr, chic.heatmap.paneled_H3K27_Somatic_CN_chr,
) %>%
  rowwise() %>%
  reframe(
    mark,
    ttest = t.test(
      germline[
        names(quartile.factor_Germline)[quartile.factor_Germline != "Q1"],
        "50%"
      ],
      somatic[
        names(quartile.factor_Somatic)[quartile.factor_Somatic != "Q1"],
        "50%"
      ]
    ) %>%
      list(),
    x = "50%",
    t = ttest$statistic["t"],
    p.adjust = ttest$p.value,
  ) %>%
  mutate(
    p.adjust = p.adjust(p.adjust),
    signif = structure(
      cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
      levels = c("****", "***", "**", "*", ""),
      class = "factor"
    )
  ) %>%
  subset(select = -ttest) %>%
  print()

# A tibble: 2 x 5
  mark     x           t p.adjust signif
  <chr>    <chr>   <dbl>    <dbl> <fct> 
1 H3K4me3  50%     0.105 9.17e- 1 ""    
2 H3K27me3 50%   -20.0   2.39e-87 "****"


## Transposable elements selected for the violin plot and their L2FC t-test

In [82]:
tar_load(
  enriched.transposable.elements.peakcalling.broad.masked
)
H3K4_Germline <- tar_read(chic.experiment.quantify_H3K4_Germline_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K4_Somatic <- tar_read(chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K27_Germline <- tar_read(chic.experiment.quantify_H3K27_Germline_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K27_Somatic <- tar_read(chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K9_Germline <- tar_read(chic.experiment.quantify_H3K9_Germline_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K9_Somatic <- tar_read(chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]

In [ ]:
plot_features <- which(
    H3K4_Germline$score.molH3 >= 0.5 &
      H3K27_Germline$score.molH3 >= 0.5 &
      H3K9_Germline$score.molH3 >= 0.5 &
      H3K4_Somatic$score.molH3 >= 0.5 &
      H3K27_Somatic$score.molH3 >= 0.5 &
      H3K9_Somatic$score.molH3 >= 0.5
  )
data <- tribble(
  ~mark, ~germline, ~somatic,
  "H3K4me3", H3K4_Germline$L2FC[plot_features], H3K4_Somatic$L2FC[plot_features],
  "H3K27me3", H3K27_Germline$L2FC[plot_features], H3K27_Somatic$L2FC[plot_features],
  "H3K9me3", H3K9_Germline$L2FC[plot_features], H3K9_Somatic$L2FC[plot_features],
) %>%
  rowwise() %>%
  reframe(
    mark,
    ttest = t.test(germline, somatic, paired=T) %>% list(),
    t = ttest$statistic["t"],
    p.adjust = ttest$p.value,
  ) %>%
  mutate(
    p.adjust = p.adjust(p.adjust),
    signif = structure(
      cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
      levels = c("****", "***", "**", "*", ""),
      class = "factor"
    )
  ) %>%
  print()

# A tibble: 3 x 5
  mark     ttest       t p.adjust signif
  <chr>    <list>  <dbl>    <dbl> <fct> 
1 H3K4me3  <htest>  3.50 1.30e- 3 **    
2 H3K27me3 <htest> -2.40 1.79e- 2 *     
3 H3K9me3  <htest>  8.55 1.52e-13 ****  


## Supplemental LOESS Model (Gene TSS)

In [1]:
tar_load(
  c(
    chic.experiment.quantify.smooth_bw40_H3K4_Germline_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K27_Germline_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K9_Germline_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K4_Somatic_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K27_Somatic_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K9_Somatic_CN_chr,
    chic.gene.tss.diameter_40_chr
  )
)

ERROR: Error in tar_load(c(chic.experiment.quantify.smooth_bw40_H3K4_Germline_CN_chr, : could not find function "tar_load"


In [5]:
tar_load(
  c(
    chic.gene.enrichment.l2fc,
    Upd_cpm
  )
)
marks <- data.frame(
  celltype = c("Germline", "Somatic") %>% factor(., .) %>% rep(each = 3 * nrow(Upd_cpm)),
  mark = c("H3K4me3", "H3K27me3", "H3K9me3") %>% factor(., .) %>% rep(each = nrow(Upd_cpm)),
  melt(chic.gene.enrichment.l2fc[7:12], variable.name = "group", value.name = "L2FC"),
  logCPM = c(
    rep(log(Upd_cpm[, "germline"]) / log(10), 3) %>%
      replace(!is.finite(.), NA),
    rep(log(Upd_cpm[, "somatic"]) / log(10), 3) %>%
      replace(!is.finite(.), NA)
  ),
  color = c(
    rep(quartile.factor_Germline, 3),
    rep(quartile.factor_Somatic, 3)
  ) %>%
    structure(levels = c("off", "low", "medium", "high"), class = "factor")
) %>%
  tibble() %>%
  print()

No id variables; using all as measure variables



# A tibble: 105,348 x 6
   celltype mark    group            L2FC   logCPM color 
   <fct>    <fct>   <fct>           <dbl>    <dbl> <fct> 
 1 Germline H3K4me3 H3K4_Germline NA       2.36    high  
 2 Germline H3K4me3 H3K4_Germline -0.450   0.374   off   
 3 Germline H3K4me3 H3K4_Germline  0.298   0.00598 off   
 4 Germline H3K4me3 H3K4_Germline  0.768   1.40    medium
 5 Germline H3K4me3 H3K4_Germline  0.210  NA       off   
 6 Germline H3K4me3 H3K4_Germline -1.03    0.806   low   
 7 Germline H3K4me3 H3K4_Germline  0.0912  0.328   off   
 8 Germline H3K4me3 H3K4_Germline  0.443  -0.294   off   
 9 Germline H3K4me3 H3K4_Germline -0.0860  0.0128  off   
10 Germline H3K4me3 H3K4_Germline  0.0784  0.794   low   
# i 105,338 more rows


In [20]:
library(ggplot2)
xlim <- c(-3, 3)
ylim <- c(-3, 5)
markloess <- marks %>%
  group_by(group) %>%
  reframe(
    y = seq(min(ylim), max(ylim), by=0.01) %>%
      round(2) %>%
      subset(between(., logCPM %>% subset(. > -6) %>% quantile(0.025), logCPM %>% subset(. > -6) %>% quantile(0.975))),
    predict(
      loess(L2FC ~ logCPM),
      newdata=y,
      se = TRUE
    ) %>%
      as_tibble() %>%
      reframe(L2FC = fit, L2FC.se = se.fit),
    logCPM = y
  ) %>%
  print(n = 3)
ggplot(markloess, aes(y, L2FC, color=group)) + geom_line()

In [21]:
log10 <- \(v) log(v) / log(10)
x <- mapply(
  \(cpm, level) cpm %>%
    split(level) %>%
    sapply(\(v) v %>% subset(v != 0) %>% median %>% log10 %>% round(digits=2)),
  as.data.frame(Upd_cpm[, 1:2]),
  list(quartile.factor_Germline, quartile.factor_Somatic)
)
data <- markloess %>%
  group_by(group) %>%
  reframe(
    celltype = tolower(str_extract(group[1], "Germline|Somatic")),
    tibble(name = c("Q2", "Q3", "Q4")) %>%
      rowwise() %>%
      mutate(
        teststat = (
          L2FC[match(x[name, celltype], y)] -
            L2FC[match(x["Q1", celltype], y)]
        ) /
          sqrt(
            L2FC.se[match(x[name, celltype], y)]^2 +
              L2FC.se[match(x["Q1", celltype], y)]
          ),
        p.adjust = teststat %>%
          abs() %>%
          pnorm(lower.tail = F) %>%
          `*`(2) %>%
          pmin(1)
      )
  ) %>%
  subset(select = c(1, 3, 4, 5)) %>%
  mutate(
    p.adjust = p.adjust(p.adjust),
    signif = structure(
      cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
      levels = c("****", "***", "**", "*", ""),
      class = "factor"
    )
  ) %>%
  print()

# A tibble: 18 x 5
   group          name  teststat p.adjust signif
   <fct>          <chr>    <dbl>    <dbl> <fct> 
 1 H3K4_Germline  Q2       4.26  2.07e- 4 "***" 
 2 H3K4_Germline  Q3       6.11  1.43e- 8 "****"
 3 H3K4_Germline  Q4       7.02  3.31e-11 "****"
 4 H3K27_Germline Q2       0.470 1   e+ 0 ""    
 5 H3K27_Germline Q3       0.599 1   e+ 0 ""    
 6 H3K27_Germline Q4       0.336 1   e+ 0 ""    
 7 H3K9_Germline  Q2      -1.94  2.59e- 1 ""    
 8 H3K9_Germline  Q3      -2.85  2.65e- 2 "*"   
 9 H3K9_Germline  Q4      -3.64  2.49e- 3 "**"  
10 H3K4_Somatic   Q2       5.82  6.97e- 8 "****"
11 H3K4_Somatic   Q3       7.46  1.52e-12 "****"
12 H3K4_Somatic   Q4       7.85  7.29e-14 "****"
13 H3K27_Somatic  Q2      -1.84  2.65e- 1 ""    
14 H3K27_Somatic  Q3      -3.63  2.49e- 3 "**"  
15 H3K27_Somatic  Q4      -5.94  3.62e- 8 "****"
16 H3K9_Somatic   Q2      -3.60  2.49e- 3 "**"  
17 H3K9_Somatic   Q3      -5.44  5.79e- 7 "****"
18 H3K9_Somatic   Q4      -7.08  2.30e-11 "****"
